### Vision input output

raw image -> numerical encoding -> model -> output -> predicted output <br>

* input shapes
```[batch_size, width, height, color_channel]``` ex: [None, 224, 224, 3], [32, 224, 224, 3]

batch_size is problem specific depending on hardware

* output shape: number of class [n, ....]

Different input shapes (for different frameworks):
1. shape -> [None, 28, 28, 1] (NHWC)
2. shape -> [None, 1, 28, 28] (NCHW)

##### Path to build
* Get the data ready (convert to tensors)
* Build or pick a model - Pick a loss function, Build a training loop
* Fit the model to the data and make prediction
* Evaluate the model
* Improve through experimentation
* Save and reload the trained model

#### CNN
- Convolutional layer - 'requires input data, filter (feature detector/kernel), featuremap'
`nb`: we have a feature detector, also known as a kernel or a filter, which will move across the receptive fields of the image, checking if the feature is present. This process is known as a convolution.
- Pooling layer
- Fully-connected layer

#### Libs
* `torchvision.datasets` - get datasets and data loading function for computer vision here
* `torchvision.models` - pytorch pre-trained modesl
* `torchvision.transforms` - functions  for manipulating vision data to be suitable for use with
* `torch.utils.data.Dataset` - Base dataset class for pytorch
* `torch.utils.data.DataLoader` - create python iterable over a

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

1. Getting dataset
`FashionMNIST`

In [ ]:
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
len(train_data), len(test_data)

In [ ]:
image, label = train_data[0]

image.shape

In [ ]:
class_names = train_data.classes
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
image, label = train_data[0]
print(f'Image size: {image.shape}')

plt.imshow(image.squeeze(), cmap='gray')
plt.title(label=class_names[label])
plt.axis(False)

In [ ]:
# plot more
torch.manual_seed(42)
fig = plt.figure(figsize=(9, 9))
rows, cols =4, 4
for i in range(1, rows*cols+1):
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    # print(random_idx)
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(class_names[label])
    plt.axis(False)

In [ ]:
train_data, test_data

#### 2. prepare dataloader

Now dataset is in form of pytorch datasets.
DataLoader turns dataset into a python iterable or turn into mini-batches

1. its more computationally efficient as in computing hardware
2. it gives our neural network more chanaces to update its gradients per epoch

dataloader: https://pytorch.org/docs/stable/data.html

In [ ]:
from torch.utils.data import DataLoader

# setup batch size hyperparameter
BATCH_SIZE = 32

# turn datasets intor iterables
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

train_dataloader, test_dataloader

In [ ]:
print(f'Dataloader: {train_dataloader, test_dataloader}')
print(f'Length: train - {len(train_dataloader)} batches of {BATCH_SIZE}')
print(f'Lenght: test - {len(test_dataloader)} batches of {BATCH_SIZE}')

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
print(train_features_batch.shape, train_labels_batch.shape)
# show a sample from dataloader
# torch.manual_seed(42)
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap='gray')
plt.title(class_names[label])
plt.axis(False)
print(f'Image size: {img.shape}')
print(f'Label: {label}, label shape: {label.shape}')

15:25